In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertModel
from sentence_transformers import SentenceTransformer
import faiss
import sentence_transformers

c:\Users\tanne\Documents\nlp_tutorial\nlp_elastic-search\64_bit_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = pd.read_json("https://lp-prod-resources.s3.amazonaws.com/493/57248/2021-05-04-13-31-46/sentences.json").rename(columns = {0:"sentence_text"})
questions = pd.read_json("https://lp-prod-resources.s3.amazonaws.com/493/57248/2021-08-16-19-04-45/questions.json").rename(columns = {0:"question_text"})
documents = sentences.sentence_text.to_list()


In [3]:
index = faiss.read_index("sbert_search_index")

In [4]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [5]:
def encode(document: str):
  vector = model.encode(document)
  return vector

In [6]:
def search(query: str, k=1):
  encoded_query = np.expand_dims(encode(query), 0)
  top_k = index.search(encoded_query, k)
  scores = top_k[0][0]
  results = [documents[_id] for _id in top_k[1][0]]
  return list(zip(results, scores))

In [17]:
q_num = 5
num_similar_docs = 1

print("Question: {}".format(questions.question_text[q_num])), print("Answer: {}".format(search(questions.question_text[q_num], num_similar_docs)))

Question: Giant viruses classification
Answer: [('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.', 73.13219)]


(None, None)

In [8]:
from transformers import pipeline
 
qNa= pipeline('question-answering', model= 'ktrapeznikov/albert-xlarge-v2-squad-v2', tokenizer='ktrapeznikov/albert-xlarge-v2-squad-v2')
 

In [18]:
paragraph = "A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people."

In [26]:
ans = qNa({'question': 'how many people are affected by a pandemic?','context': f'{paragraph}'})
print(ans)

{'score': 0.5302037000656128, 'start': 146, 'end': 167, 'answer': ' a substantial number'}
